# Processamento dos dados

In [1]:
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.9


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import pandas as pd
import numpy as np
import sqlite3
import re
from IPython.display import display
from zipfile import ZipFile  

pd.options.display.max_columns = None

## Dados das Ocorrências
Partindo dos dados de todas as ocorrências registradas no Compras Governamentais, vamos obter o número de ocorrências por fornecedor, excluindo as ocorrências que não tem relação com execução contratual.

In [4]:
ocorrencias = pd.read_csv('./ocorrencias.csv', encoding = 'latin-1')

In [5]:
ocorrencias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260012 entries, 0 to 260011
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Id                    260012 non-null  int64 
 1   CNPJ                  248333 non-null  object
 2   CPF                   551 non-null     object
 3   Tipo pessoa           248884 non-null  object
 4   Número do Processo    260011 non-null  object
 5   Unidade Cadastradora  260012 non-null  object
 6   Tipo de Ocorrência    260012 non-null  object
dtypes: int64(1), object(6)
memory usage: 13.9+ MB


Não vamos utilizar os dados dos fornecedores Pessoa Física, já que os CPFs estão mascarados e nosso interesse, na presente análise, está nas Pessoas Jurídicas, que são a imensa maioria dos fornecedores do Governo Federal.

In [6]:
ocorrencias.dropna(subset = ['CPF']).dropna(axis = 1, how = 'all').head() #Exibindo alguns exemplos de CPFs mascarados

,Id,CPF,Tipo pessoa,Número do Processo,Unidade Cadastradora,Tipo de Ocorrência
375,376,Fornecedor ***725278**: RONALD REMONDY JUNIOR,PF,03111003027200770,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
4224,4226,Fornecedor ***429031**: THIAGO SILVEIRA CORREIA,PF,230000590132007,153234: ESCOLA AGROTECNICA FEDERAL DE CERES/GO,7: Outros Tipos de Ocorrência
8183,8190,Fornecedor ***822201**: ROBERT FAGNER INACIO ...,PF,00001/2008,160101: 23 COMPANHIA DE ENGENHARIA DE COMBATE-...,"1: Advertência - Lei nº 8666/93, art. 87, inc. I"
8331,8338,Fornecedor ***219500**: ALEXANDRE RAFAEL LOTT...,PF,DISP 126/2010,160435: 7 REGIMENTO DE CAVALARIA MECANIZADO/RS,6: Inativação a Pedido do Fornecedor
16886,16911,Fornecedor ***821916**: JOAO PAULO DIAS,PF,2003MAUA007,160106: 2 BATALHAO FERROVIáRIO,7: Outros Tipos de Ocorrência


In [7]:
ocorrencias['Tipo de Ocorrência'].unique()

array(['7: Outros Tipos de Ocorrência', '8: Legado',
       '1: Advertência - Lei nº 8666/93, art. 87, inc. I',
       '2: Multa - Lei nº 8666/93, art. 87, inc. II',
       '6: Inativação a Pedido do Fornecedor',
       '5: Impedimento de Licitar e Contratar - Lei nº 10.520/02, art. 7º',
       '3: Suspensão Temporária - Lei nº 8666/93, art. 87, inc. III',
       '4: Declaração de Inidoneidade - Lei nº 8666/93, art. 87, inc. IV',
       '9: Reativação a Pedido do Fornecedor', '10: Dinâmica'],
      dtype=object)

Os tipos de ocorrência _Inativação a Pedido do Fornecedor_ e _Reativação a Pedido do Fornecedor_ não tem relação com problemas na execução contratual, então vamos excluir as linhas que se referem a esses tipos de ocorrências, deixando apenas as de interesse.

In [8]:
ocorrenciasInteresse = ['1: Advertência - Lei nº 8666/93, art. 87, inc. I',
                        '2: Multa - Lei nº 8666/93, art. 87, inc. II',
                        '3: Suspensão Temporária - Lei nº 8666/93, art. 87, inc. III',
                        '4: Declaração de Inidoneidade - Lei nº 8666/93, art. 87, inc. IV',
                        '5: Impedimento de Licitar e Contratar - Lei nº 10.520/02, art. 7º',
                        '7: Outros Tipos de Ocorrência', 
                        '8: Legado',
                        '10: Dinâmica']

ocorrencias = ocorrencias[ocorrencias['Tipo de Ocorrência'].isin(ocorrenciasInteresse)]
ocorrencias.head()

,Id,CNPJ,CPF,Tipo pessoa,Número do Processo,Unidade Cadastradora,Tipo de Ocorrência
0,1,NaN,NaN,NaN,00001005825200281,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
1,2,NaN,NaN,NaN,03110018451201024,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
2,3,NaN,NaN,NaN,03110006265201042,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
3,4,NaN,NaN,NaN,04500004532200982,200999: MINISTÉRIO DA ECONOMIA,8: Legado
4,5,NaN,NaN,NaN,03000004162200951,200999: MINISTÉRIO DA ECONOMIA,8: Legado


In [9]:
ocorrencias['Tipo de Ocorrência'].unique()

array(['7: Outros Tipos de Ocorrência', '8: Legado',
       '1: Advertência - Lei nº 8666/93, art. 87, inc. I',
       '2: Multa - Lei nº 8666/93, art. 87, inc. II',
       '5: Impedimento de Licitar e Contratar - Lei nº 10.520/02, art. 7º',
       '3: Suspensão Temporária - Lei nº 8666/93, art. 87, inc. III',
       '4: Declaração de Inidoneidade - Lei nº 8666/93, art. 87, inc. IV',
       '10: Dinâmica'], dtype=object)

Excluindo as linhas que não se referem a pessoas jurídicas:

In [10]:
ocorrencias = ocorrencias.dropna(subset = ['CNPJ']).dropna(axis = 1, how = 'all')

In [11]:
ocorrencias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247840 entries, 124 to 260011
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Id                    247840 non-null  int64 
 1   CNPJ                  247840 non-null  object
 2   Tipo pessoa           247840 non-null  object
 3   Número do Processo    247840 non-null  object
 4   Unidade Cadastradora  247840 non-null  object
 5   Tipo de Ocorrência    247840 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.2+ MB


In [12]:
ocorrencias.head()

,Id,CNPJ,Tipo pessoa,Número do Processo,Unidade Cadastradora,Tipo de Ocorrência
124,125,Fornecedor 00.725.347/0001-00: CONSTRUTORA GA...,PJ,04300006506200783,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
238,239,Fornecedor 03.707.204/0001-55: C.R.O. CONSTRU...,PJ,03111001829200826,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
675,676,Fornecedor 32.952.822/0001-69: TERTEC CONSTRU...,PJ,04300004855200761,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
929,930,Fornecedor 88.674.080/0001-01: TRAMONTINA ELE...,PJ,04300004293200411,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
957,958,Fornecedor 00.000.000/0001-91: BANCO DO BRASI...,PJ,63015375/201019,791180: BASE AEREA NAVAL DE SAO PEDRO DA ALDEI...,"1: Advertência - Lei nº 8666/93, art. 87, inc. I"


Na coluna CNPJ, queremos deixar apenas o número do CNPJ, sem a palavra Fornecedor e o sem o nome da empresa.

In [13]:
ocorrencias['CNPJ'] = ocorrencias['CNPJ'].str.split(":", n = 1, expand = True)[0]
ocorrencias['CNPJ'] = ocorrencias['CNPJ'].apply(lambda x: re.sub('[^0-9]', '', x))

In [14]:
ocorrencias.head()

,Id,CNPJ,Tipo pessoa,Número do Processo,Unidade Cadastradora,Tipo de Ocorrência
124,125,00725347000100,PJ,04300006506200783,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
238,239,03707204000155,PJ,03111001829200826,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
675,676,32952822000169,PJ,04300004855200761,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
929,930,88674080000101,PJ,04300004293200411,200999: MINISTÉRIO DA ECONOMIA,7: Outros Tipos de Ocorrência
957,958,00000000000191,PJ,63015375/201019,791180: BASE AEREA NAVAL DE SAO PEDRO DA ALDEI...,"1: Advertência - Lei nº 8666/93, art. 87, inc. I"


A seguir, contamos quantas ocorrencias existem para cada CNPJ.

In [15]:
quantidadePorCnpj = {}
for cnpj in ocorrencias['CNPJ'].unique():
    numeroOcorrencias = len(ocorrencias[ocorrencias['CNPJ'] == cnpj])
    if cnpj not in quantidadePorCnpj:
        quantidadePorCnpj[cnpj] = numeroOcorrencias
    else:
        quantidadePorCnpj[cnpj] = quantidadePorCnpj[cnpj] + numeroOcorrencias

In [16]:
ocorrenciasPorFornecedor = pd.DataFrame.from_dict(quantidadePorCnpj, dtype = 'int64', orient = 'index', columns = ['ocorrencias'])

In [17]:
ocorrenciasPorFornecedor.reset_index(inplace = True)

In [18]:
ocorrenciasPorFornecedor.rename(columns = {'index': 'CNPJ'}, inplace = True)

In [19]:
ocorrenciasPorFornecedor.head()

,CNPJ,ocorrencias
0,00725347000100,6
1,03707204000155,2
2,32952822000169,1
3,88674080000101,1
4,00000000000191,8


Verificando se todos os CNPJs são únicos (número de linhas deve ser igual à quantidade de valores únicos):

In [20]:
ocorrenciasPorFornecedor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39730 entries, 0 to 39729
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CNPJ         39730 non-null  object
 1   ocorrencias  39730 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 620.9+ KB


In [21]:
ocorrenciasPorFornecedor['CNPJ'].nunique()

39730

In [22]:
ocorrenciasPorFornecedor.to_csv('./ocorrenciasPorFornecedor.csv', index = False)

## Dados dos Contratos
A partir dos dados dos contratos, vamos obter os CNPJs dos contratados pelo Governo Federal. Consideraremos apenas as contratações realizadas por UGs que registraram pelo menos uma ocorrência no sistema.

In [26]:
ugs = ocorrencias['Unidade Cadastradora'].str.split(pat = ':', expand = True)

In [27]:
ugs.isnull().sum()

0    0
1    0
dtype: int64

In [28]:
ugs.tail()

,0,1
260007,925936,ECT - DIRETORIA REGIONAL SAO PAULO METROPOLIT
260008,925936,ECT - DIRETORIA REGIONAL SAO PAULO METROPOLIT
260009,148002,CORREIOS SEDE
260010,148002,CORREIOS SEDE
260011,253002,AGENCIA NACIONAL DE VIGILANCIA SANITARIA - DF


Conforme o [dicionário de dados dos contratos](http://www.portaltransparencia.gov.br/pagina-interna/603396-dicionario-de-dados-contratos-firmados), vemos que cada arquivo _.zip_ contém quatro arquivos para o mês. A partir das tabelas _Compras_ podemos obter quantos contratos o governo firmou com cada fornecedor.

In [23]:
pasta = './contratos/'
for arquivoZip in os.listdir(pasta):
    arquivoParaExtrair = arquivoZip.replace('.zip', '_Compras.csv')
    ZipFile(pasta + arquivoZip).extractall(pasta + 'contratados/')

# Excluindo os arquivos que não serão usados
pasta = './contratos/contratados/'
for arquivo in os.listdir(pasta):
    if 'Compras' not in arquivo:
        os.remove(pasta + arquivo) 

Abaixo, vamos abrir cada arquivo da pasta e contar os contratos de cada fornecedor que aparece no arquivo, e em seguida, incluir as informações num dicionário. Ao final do processamento de todos os arquivos, criaremos um DataFrame a partir do dicionário.

In [29]:
contratosFornecedor = {}

In [30]:
pasta = './contratos/contratados/'
for arquivo in os.listdir(pasta):
    print(arquivo)
    for contratos in pd.read_csv(pasta + arquivo, sep = ';', dtype = 'str', encoding = 'latin-1', chunksize = 1000):
        contratos.drop(contratos.loc[~contratos['Código UG'].isin(ugs[0])].index, inplace = True)
        for cnpjFornecedor in contratos['CNPJ Contratado'].unique():
            quantidadeContratos = len(contratos[contratos['CNPJ Contratado'] == cnpjFornecedor])
            if cnpjFornecedor not in contratosFornecedor:
                contratosFornecedor[cnpjFornecedor] = quantidadeContratos
            else:
                quantidadeContratosAnterior = contratosFornecedor[cnpjFornecedor]
                contratosFornecedor[cnpjFornecedor] = quantidadeContratosAnterior + quantidadeContratos

201301_Compras.csv
201302_Compras.csv
201303_Compras.csv
201304_Compras.csv
201305_Compras.csv
201306_Compras.csv
201307_Compras.csv
201308_Compras.csv
201309_Compras.csv
201310_Compras.csv
201311_Compras.csv
201312_Compras.csv
201401_Compras.csv
201402_Compras.csv
201403_Compras.csv
201404_Compras.csv
201405_Compras.csv
201406_Compras.csv
201407_Compras.csv
201408_Compras.csv
201409_Compras.csv
201410_Compras.csv
201411_Compras.csv
201412_Compras.csv
201501_Compras.csv
201502_Compras.csv
201503_Compras.csv
201504_Compras.csv
201505_Compras.csv
201506_Compras.csv
201507_Compras.csv
201508_Compras.csv
201509_Compras.csv
201510_Compras.csv
201511_Compras.csv
201512_Compras.csv
201601_Compras.csv
201602_Compras.csv
201603_Compras.csv
201604_Compras.csv
201605_Compras.csv
201606_Compras.csv
201607_Compras.csv
201608_Compras.csv
201609_Compras.csv
201610_Compras.csv
201611_Compras.csv
201612_Compras.csv
201701_Compras.csv
201702_Compras.csv
201703_Compras.csv
201704_Compras.csv
201705_Compr

In [31]:
contratosPorFornecedor = pd.DataFrame.from_dict(contratosFornecedor, orient = 'index', columns = ['contratos'])

In [32]:
contratosPorFornecedor.reset_index(inplace = True)

In [33]:
contratosPorFornecedor.rename(columns = {'index': 'CNPJ'}, inplace = True)

In [34]:
contratosPorFornecedor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63135 entries, 0 to 63134
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CNPJ       63135 non-null  object
 1   contratos  63135 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 986.6+ KB


In [35]:
contratosPorFornecedor['CNPJ'].nunique()

63135

In [36]:
contratosPorFornecedor.head()

,CNPJ,contratos
0,04358183000172,4
1,05120923000109,60
2,07918078000139,1
3,00212655000123,270
4,05156333000128,15


Nos dados de contratos baixados do Portal da Transparência constam informações de todos os contratados, inclusive os que são Pessoas Físicas e empresas estrangeiras que não possuem CNPJ. Vamos verificar quantas das linhas da coluna cnpj não contém CNPJs:

Removendo as letras:

In [37]:
contratosPorFornecedor['CNPJ'] = contratosPorFornecedor['CNPJ'].apply(lambda x: re.sub('[^0-9]', '', x))

Contando quantas linhas contém CNPJs e quantas não:

In [38]:
print('CNPJs: ' + str(contratosPorFornecedor['CNPJ'].loc[contratosPorFornecedor['CNPJ'].str.len() == 14].count()))
print('Outros: ' + str(contratosPorFornecedor['CNPJ'].loc[contratosPorFornecedor['CNPJ'].str.len() != 14].count()))
print('Total de linhas: '  + str(len(contratosPorFornecedor)))

CNPJs: 56855
Outros: 6280
Total de linhas: 63135


Vemos que existem 6.280 contratados que não são pessoas jurídicas. Vamos excluir estas linhas e trabalhar apenas com os fornecedores PJ, que são o nosso foco nesta análise.

In [39]:
contratosPorFornecedor.drop(contratosPorFornecedor['CNPJ'].loc[contratosPorFornecedor['CNPJ'].str.len() != 14].index, inplace = True)

In [40]:
contratosPorFornecedor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56855 entries, 0 to 63134
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CNPJ       56855 non-null  object
 1   contratos  56855 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [41]:
contratosPorFornecedor.to_csv('./contratosPorFornecedor.csv', index = False)

## Dados das Licitações
A partir dos dados das licitações, vamos obter os CNPJs dos fornecedores do Governo Federal e o respectivo número de participações e vitórias. Consideraremos apenas participações e vitórias em licitações realizadas por UGs que registraram pelo menos uma ocorrência no sistema e os CNPJs de empresas que firmaram pelo menos um contrato com o Governo Federal.

In [42]:
cnpjs = contratosPorFornecedor['CNPJ'] 

In [43]:
cnpjs

0        04358183000172
1        05120923000109
2        07918078000139
3        00212655000123
4        05156333000128
              ...      
63130    16611560000112
63131    37045935000102
63132    84084383000113
63133    18376070000104
63134    35613306000106
Name: CNPJ, Length: 56855, dtype: object

Conforme o [dicionário de dados das licitações](http://www.portaltransparencia.gov.br/pagina-interna/603389-dicionario-de-dados-licitacoes), vemos que cada arquivo _.zip_ contém três arquivos para o mês (_ItensLicitação_, _Licitação_ e _ParticipantesLicitação_). Na tabela _ParticipantesLicitação_ temos, para cada item de cada licitação, os fornecedores que concorreram e qual ganhou (através de _Flag vencedor_). Portanto, a partir dessas tabelas _ParticipantesLicitacao_ podemos obter, para cada fornecedor, a quantidade de itens que ele concorreu e a quantidade de itens que venceu.

In [44]:
pasta = './licitacoes/'
for arquivoZip in os.listdir(pasta):
    arquivoParaExtrair = arquivoZip.replace('.zip', '_ParticipantesLicitação.csv')
    ZipFile(pasta + arquivoZip).extractall(pasta + 'participantes/')

# Excluindo os arquivos que não serão usados
pasta = './licitacoes/participantes/'
for arquivo in os.listdir(pasta):
    if 'Participantes' not in arquivo:
        os.remove(pasta + arquivo) 

Abaixo, vamos abrir cada arquivo da pasta e contar as participações e vitórias de cada fornecedor que aparece no arquivo, e em seguida, incluir as informações num dicionário. Ao final do processamento de todos os arquivos, criaremos um DataFrame a partir do dicionário. Como alguns arquivos possuem muitas linhas (em alguns casos quase 1,5 milhão de linhas), a obtenção dos CNPJs únicos e do número de vitórias para cada CNPJ pode demorar muito, então vamos ler cada arquivo e fazer a contagem por partes. 

In [45]:
itensPorFornecedor = {}

In [46]:
pasta = './licitacoes/participantes/'
for arquivo in os.listdir(pasta):
    print(arquivo)
    for itens in pd.read_csv(pasta + arquivo, sep = ';', dtype = 'str', encoding = 'latin-1', chunksize = 1000):
        itens.drop(itens.loc[~itens['Código UG'].isin(ugs[0])].index, inplace = True)
        itens.drop(itens.loc[~itens['CNPJ Participante'].isin(cnpjs)].index, inplace = True)
        for cnpjFornecedor in itens['CNPJ Participante'].unique():
            itensParticipacao = itens[itens['CNPJ Participante'] == cnpjFornecedor]
            quantidadeParticipacoes = len(itensParticipacao)
            quantidadeVitorias = len(itensParticipacao[itensParticipacao['Flag Vencedor'] == 'SIM'])
            if cnpjFornecedor not in itensPorFornecedor:
                itensPorFornecedor[cnpjFornecedor] = [quantidadeParticipacoes, quantidadeVitorias]
            else:
                quantPartAnterior = itensPorFornecedor[cnpjFornecedor][0]
                quantVitAnterior = itensPorFornecedor[cnpjFornecedor][1]
                itensPorFornecedor[cnpjFornecedor] = [quantPartAnterior + quantidadeParticipacoes, quantVitAnterior + quantidadeVitorias]

201301_ParticipantesLicitaç╞o.csv
201302_ParticipantesLicitaç╞o.csv
201303_ParticipantesLicitaç╞o.csv
201304_ParticipantesLicitaç╞o.csv
201305_ParticipantesLicitaç╞o.csv
201306_ParticipantesLicitaç╞o.csv
201307_ParticipantesLicitaç╞o.csv
201308_ParticipantesLicitaç╞o.csv
201309_ParticipantesLicitaç╞o.csv
201310_ParticipantesLicitaç╞o.csv
201311_ParticipantesLicitaç╞o.csv
201312_ParticipantesLicitaç╞o.csv
201401_ParticipantesLicitaç╞o.csv
201402_ParticipantesLicitaç╞o.csv
201403_ParticipantesLicitaç╞o.csv
201404_ParticipantesLicitaç╞o.csv
201405_ParticipantesLicitaç╞o.csv
201406_ParticipantesLicitaç╞o.csv
201407_ParticipantesLicitaç╞o.csv
201408_ParticipantesLicitaç╞o.csv
201409_ParticipantesLicitaç╞o.csv
201410_ParticipantesLicitaç╞o.csv
201411_ParticipantesLicitaç╞o.csv
201412_ParticipantesLicitaç╞o.csv
201501_ParticipantesLicitaç╞o.csv
201502_ParticipantesLicitaç╞o.csv
201503_ParticipantesLicitaç╞o.csv
201504_ParticipantesLicitaç╞o.csv
201505_ParticipantesLicitaç╞o.csv
201506_Partici

In [47]:
licitacoesPorFornecedor = pd.DataFrame.from_dict(itensPorFornecedor, orient = 'index', columns = ['participacoes', 'vitorias'])

In [48]:
licitacoesPorFornecedor.reset_index(inplace = True)

In [49]:
licitacoesPorFornecedor.rename(columns = {'index': 'CNPJ'}, inplace = True)

In [50]:
licitacoesPorFornecedor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52749 entries, 0 to 52748
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CNPJ           52749 non-null  object
 1   participacoes  52749 non-null  int64 
 2   vitorias       52749 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


In [51]:
licitacoesPorFornecedor['CNPJ'].nunique()

52749

Conferindo se todas as linhas contém CNPJs:

In [54]:
print('CNPJs: ' + str(licitacoesPorFornecedor['CNPJ'].loc[licitacoesPorFornecedor['CNPJ'].str.len() == 14].count()))
print('Outros: ' + str(licitacoesPorFornecedor['CNPJ'].loc[licitacoesPorFornecedor['CNPJ'].str.len() != 14].count()))
print('Total de linhas: '  + str(len(licitacoesPorFornecedor)))

CNPJs: 52749
Outros: 0
Total de linhas: 52749


In [55]:
licitacoesPorFornecedor.to_csv('./licitacoesPorFornecedor.csv', index = False)

## Dados da base do CNPJ

In [56]:
baseCnpj = sqlite3.connect('./qsacnpj/bd_dados_qsa_cnpj.db')
cursor = baseCnpj.cursor()

Verificando quais as tabelas que existem no banco:

In [57]:
tabelas = cursor.execute('SELECT name FROM sqlite_master WHERE type="table"').fetchall()
tabelas

[('cnpj_dados_cadastrais_pj',),
 ('cnpj_dados_socios_pj',),
 ('cnpj_dados_cnae_secundario_pj',),
 ('tab_cnpj_entes_publicos',),
 ('tab_qualificacao_responsavel_socio',),
 ('tab_situacao_cadastral',),
 ('tab_natureza_juridica',),
 ('tab_cnae',),
 ('tab_codigo_municipios_siafi',)]

Vamos buscar alguns registros da tabela dos dados cadastrais de CNPJ para verificar que informações estão disponíveis.

In [58]:
df = pd.read_sql('SELECT * FROM cnpj_dados_cadastrais_pj LIMIT 100', baseCnpj)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   tipo_de_registro             100 non-null    object 
 1   indicador                    100 non-null    object 
 2   tipo_atualizacao             100 non-null    object 
 3   cnpj                         100 non-null    object 
 4   identificador_matriz_filial  100 non-null    object 
 5   razao_social                 100 non-null    object 
 6   nome_fantasia                100 non-null    object 
 7   situacao_cadastral           100 non-null    object 
 8   data_situacao_cadastral      100 non-null    object 
 9   motivo_situacao_cadastral    100 non-null    object 
 10  nm_cidade_exterior           100 non-null    object 
 11  cod_pais                     100 non-null    object 
 12  nm_pais                      100 non-null    object 
 13  codigo_natureza_jurid

In [60]:
df.head()

,tipo_de_registro,indicador,tipo_atualizacao,cnpj,identificador_matriz_filial,razao_social,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nm_cidade_exterior,cod_pais,nm_pais,codigo_natureza_juridica,data_inicio_atividade,cnae_fiscal,descricao_tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,codigo_municipio,municipio,ddd_telefone_1,ddd_telefone_2,ddd_fax,correio_eletronico,qualificacao_responsavel,capital_social_empresa,porte_empresa,opcao_pelo_simples,data_opcao_pelo_simples,data_exclusao_simples,opcao_pelo_mei,situacao_especial,data_situacao_especial,filler,fim_registro
0,1,F,,00000000000191,1,BANCO DO BRASIL SA,DIRECAO GERAL,02,2005-11-03,00,,,,2038,1966-08-01,6422100,QUADRA,"SAUN QUADRA 5 LOTE B TORRES I, II E III",SN,ANDAR 1 A 16 SALA 101 A 1601 ...,ASA NORTE,70040912,DF,9701,BRASILIA,61 34939002,,61 34931040,SECEX@BB.COM.BR,10,9.000000e+10,05,0,None,None,N,,None,,F
1,1,F,,00000000000272,2,BANCO DO BRASIL SA,MANAUS (AM),02,2005-11-03,00,,,,2038,1976-08-13,6422100,RUA,GUILHERME MOREIRA,315,,CENTRO,69005300,AM,0255,MANAUS,92 36215500,,,AGE0002@BB.COM.BR,10,0.000000e+00,05,0,None,None,,,None,,F
2,1,F,,00000000000353,2,BANCO DO BRASIL SA,PRESIDENTE VARGAS BELEM (PA),02,2005-11-03,00,,,,2038,1966-08-01,6422100,,AVEN PRESIDENTE VARGAS,248,,COMERCIO,66010900,PA,0427,BELEM,,,,,10,0.000000e+00,05,0,None,None,,,None,,F
3,1,F,,00000000000434,2,BANCO DO BRASIL SA,SANTOS - SANTOS (SP),02,2005-11-03,00,,,,2038,1966-08-01,6422100,RUA,15 DE NOVEMBRO,195,,CENTRO,11010908,SP,7071,SANTOS,,,,,10,0.000000e+00,05,0,None,None,,,None,,F
4,1,F,,00000000000515,2,BANCO DO BRASIL SA,CAMPOS EST.UNIF.,02,2005-11-03,00,,,,2038,1966-08-01,6421200,PRACA,DAS 4 JORNADAS,11,,CENTRO,28030002,RJ,5819,CAMPOS DOS GOYTACAZES,,,,,10,0.000000e+00,05,0,None,None,,,None,,F


Segundo o [Layout de dados da RFB](http://200.152.38.155/CNPJ/LAYOUT_DADOS_ABERTOS_CNPJ.pdf), temos os seguintes significados dos códigos das colunas que não estão detalhados em outras tabelas no banco: 
###### identificador_matriz_filial
* 1: Matriz
* 2: Filial

###### porte_empresa
* 00: Não informado
* 01: Micro Empresa
* 03: Empresa de Pequeno Porte
* 05: Demais

###### opcao_pelo_simples
* 0 ou em branco - não optante
* 5 e 7: optante pelo SIMPLES
* 6 e 8: excluído do SIMPLES

###### situacao_cadastral
* 01: Nula
* 02: Ativa
* 03: Suspensa
* 04: Inapta
* 08: Baixada

As naturezas jurídicas estão descritas na tabela _tab_natureza_juridica_:

In [61]:
natJurid = pd.read_sql('SELECT * FROM tab_natureza_juridica', baseCnpj)
natJurid

,cod_natureza_juridica,nm_natureza_juridica,cod_subclass_natureza_juridica,nm_subclass_natureza_juridica
0,1,Administração Pública,1015,Órgão Público do Poder Executivo Federal
1,1,Administração Pública,1023,Órgão Público do Poder Executivo Estadual ou d...
2,1,Administração Pública,1031,Órgão Público do Poder Executivo Municipal
3,1,Administração Pública,1040,Órgão Público do Poder Legislativo Federal
4,1,Administração Pública,1058,Órgão Público do Poder Legislativo Estadual ou...
...,...,...,...,...
84,4,Pessoas Físicas,4111,Leiloeiro
85,4,Pessoas Físicas,4120,Produtor Rural (Pessoa Física)
86,5,Organizações Internacionais e Outras Instituiç...,5010,Organização Internacional
87,5,Organizações Internacionais e Outras Instituiç...,5029,Representação Diplomática Estrangeira


A tabela _tab_cnae_ contém a descrição específica do CNAE (_nm_cnae_), mas também as informações sobre _seção_, _divisão_, _grupo_ e _classe_.

In [62]:
cnaes = pd.read_sql('SELECT * FROM tab_cnae', baseCnpj)
cnaes

,cod_secao,nm_secao,cod_divisao,nm_divisao,cod_grupo,nm_grupo,cod_classe,nm_classe,cod_cnae,nm_cnae
0,A,AGRICULTURA PECUÁRIA PRODUÇÃO FLORESTAL PESCA ...,01,AGRICULTURA PECUÁRIA E SERVIÇOS RELACIONADOS,01.1,Produção de lavouras temporárias,01.11-3,Cultivo de cereais,0111301,Cultivo de arroz
1,A,AGRICULTURA PECUÁRIA PRODUÇÃO FLORESTAL PESCA ...,01,AGRICULTURA PECUÁRIA E SERVIÇOS RELACIONADOS,01.1,Produção de lavouras temporárias,01.11-3,Cultivo de cereais,0111302,Cultivo de milho
2,A,AGRICULTURA PECUÁRIA PRODUÇÃO FLORESTAL PESCA ...,01,AGRICULTURA PECUÁRIA E SERVIÇOS RELACIONADOS,01.1,Produção de lavouras temporárias,01.11-3,Cultivo de cereais,0111303,Cultivo de trigo
3,A,AGRICULTURA PECUÁRIA PRODUÇÃO FLORESTAL PESCA ...,01,AGRICULTURA PECUÁRIA E SERVIÇOS RELACIONADOS,01.1,Produção de lavouras temporárias,01.11-3,Cultivo de cereais,0111399,Cultivo de outros cereais não especificados an...
4,A,AGRICULTURA PECUÁRIA PRODUÇÃO FLORESTAL PESCA ...,01,AGRICULTURA PECUÁRIA E SERVIÇOS RELACIONADOS,01.1,Produção de lavouras temporárias,01.12-1,Cultivo de algodão herbáceo e de outras fibras...,0112101,Cultivo de algodão herbáceo
...,...,...,...,...,...,...,...,...,...,...
1326,S,OUTRAS ATIVIDADES DE SERVIÇOS,96,OUTRAS ATIVIDADES DE SERVIÇOS PESSOAIS,96.0,Outras atividades de serviços pessoais,96.09-2,Atividades de serviços pessoais não especifica...,9609206,Serviços de tatuagem e colocação de piercing
1327,S,OUTRAS ATIVIDADES DE SERVIÇOS,96,OUTRAS ATIVIDADES DE SERVIÇOS PESSOAIS,96.0,Outras atividades de serviços pessoais,96.09-2,Atividades de serviços pessoais não especifica...,9609207,Alojamento de animais domésticos
1328,S,OUTRAS ATIVIDADES DE SERVIÇOS,96,OUTRAS ATIVIDADES DE SERVIÇOS PESSOAIS,96.0,Outras atividades de serviços pessoais,96.09-2,Atividades de serviços pessoais não especifica...,9609208,Higiene e embelezamento de animais domésticos
1329,S,OUTRAS ATIVIDADES DE SERVIÇOS,96,OUTRAS ATIVIDADES DE SERVIÇOS PESSOAIS,96.0,Outras atividades de serviços pessoais,96.09-2,Atividades de serviços pessoais não especifica...,9609299,Outras atividades de serviços pessoais não esp...


Estamos focando a análise nas empresas que firmaram contratos com o Governo Federal, ou seja, nos CNPJs que constam nos dados de contratos obtidos no Portal da Transparência. Sendo assim, vamos selecionar no banco de dados apenas as informações sobre estes fornecedores.

In [63]:
cnpjs = str(contratosPorFornecedor['CNPJ'].to_list()).replace('[', '(').replace(']', ') ')

In [64]:
query = """
        WITH fornecedores AS
        (   
            SELECT *
            FROM cnpj_dados_cadastrais_pj        
            WHERE cnpj_dados_cadastrais_pj.cnpj IN """ + cnpjs + """
        )
        SELECT fornecedores.cnpj, 
               fornecedores.identificador_matriz_filial, 
               fornecedores.razao_social,
               fornecedores.situacao_cadastral,
               fornecedores.data_inicio_atividade,
               fornecedores.uf,
               fornecedores.codigo_municipio,
               fornecedores.municipio, 
               fornecedores.capital_social_empresa,
               fornecedores.porte_empresa,
               fornecedores.opcao_pelo_simples,
               tab_natureza_juridica.*,
               tab_cnae.* 
        FROM fornecedores
        JOIN tab_natureza_juridica, tab_cnae 
        ON fornecedores.codigo_natureza_juridica = tab_natureza_juridica.cod_subclass_natureza_juridica
        AND fornecedores.cnae_fiscal = tab_cnae.cod_cnae
        """

dadosCadastroCnpj = pd.read_sql(query, baseCnpj)

In [65]:
baseCnpj.close()

In [66]:
dadosCadastroCnpj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56850 entries, 0 to 56849
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   cnpj                            56850 non-null  object 
 1   identificador_matriz_filial     56850 non-null  object 
 2   razao_social                    56850 non-null  object 
 3   situacao_cadastral              56850 non-null  object 
 4   data_inicio_atividade           56850 non-null  object 
 5   uf                              56850 non-null  object 
 6   codigo_municipio                56850 non-null  object 
 7   municipio                       56850 non-null  object 
 8   capital_social_empresa          56850 non-null  float64
 9   porte_empresa                   56850 non-null  object 
 10  opcao_pelo_simples              56850 non-null  object 
 11  cod_natureza_juridica           56850 non-null  object 
 12  nm_natureza_juridica            

Foram recuperadas 56.850 linhas do banco de dados, porém no DataFrame _contratosPorFornecedor_ existem 56.855 CNPJs diferentes. Por algum motivo 5 CNPJs não foram localizados na base de dados. Como os não encontrados são poucos comparados ao tamanho do DataFrame, vamos desconsiderá-los no momento de juntar os dataframes.

Incluindo os significados dos códigos nas colunas que faltam:

In [67]:
dadosCadastroCnpj['identificador_matriz_filial'] = pd.Series(np.where(dadosCadastroCnpj['identificador_matriz_filial'].values == '1', 'MATRIZ', 'FILIAL'), dadosCadastroCnpj.index)

In [68]:
sit = { '01': 'Nula',
        '02': 'Ativa',
        '03': 'Suspensa',
        '04': 'Inapta',
        '08': 'Baixada' }

def situacaoCadastral(linha):
    return sit.get(linha['situacao_cadastral'], 'Não encontrada')

dadosCadastroCnpj['situacao_cadastral'] = dadosCadastroCnpj.apply(lambda linha: situacaoCadastral(linha), axis = 1)

In [69]:
porte = { '00': 'Não informado',
          '01': 'Micro Empresa',
          '03': 'Empresa de Pequeno Porte',
          '05': 'Demais' }

def porteEmpresa(linha):
    return porte.get(linha['porte_empresa'], 'Não encontrado')

dadosCadastroCnpj['porte_empresa'] = dadosCadastroCnpj.apply(lambda linha: porteEmpresa(linha), axis = 1)

In [70]:
simples = { '0': 'Não optante',
            '5': 'Optante',
            '7': 'Optante',
            '6': 'Excluída',
            '8': 'Excluída'}

def opcaoSimples(linha):
    return simples.get(linha['opcao_pelo_simples'], 'Não optante')

dadosCadastroCnpj['opcao_pelo_simples'] = dadosCadastroCnpj.apply(lambda linha: opcaoSimples(linha), axis = 1)

In [71]:
dadosCadastroCnpj.head()

,cnpj,identificador_matriz_filial,razao_social,situacao_cadastral,data_inicio_atividade,uf,codigo_municipio,municipio,capital_social_empresa,porte_empresa,opcao_pelo_simples,cod_natureza_juridica,nm_natureza_juridica,cod_subclass_natureza_juridica,nm_subclass_natureza_juridica,cod_secao,nm_secao,cod_divisao,nm_divisao,cod_grupo,nm_grupo,cod_classe,nm_classe,cod_cnae,nm_cnae
0,00000000000191,MATRIZ,BANCO DO BRASIL SA,Ativa,1966-08-01,DF,9701,BRASILIA,9.000000e+10,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.22-1,Bancos múltiplos com carteira comercial,6422100,Bancos múltiplos com carteira comercial
1,00000000096369,FILIAL,BANCO DO BRASIL SA,Ativa,1975-08-11,RN,1619,APODI,0.000000e+00,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.21-2,Bancos comerciais,6421200,Bancos comerciais
2,00000000105104,FILIAL,BANCO DO BRASIL SA,Ativa,1976-03-17,RJ,6001,RIO DE JANEIRO,0.000000e+00,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.21-2,Bancos comerciais,6421200,Bancos comerciais
3,00000000304492,FILIAL,BANCO DO BRASIL SA,Ativa,1991-04-12,SP,6477,GUARULHOS,0.000000e+00,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.22-1,Bancos múltiplos com carteira comercial,6422100,Bancos múltiplos com carteira comercial
4,00000000314455,FILIAL,BANCO DO BRASIL SA,Ativa,1991-04-02,DF,9701,BRASILIA,0.000000e+00,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.22-1,Bancos múltiplos com carteira comercial,6422100,Bancos múltiplos com carteira comercial


In [72]:
dadosCadastroCnpj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56850 entries, 0 to 56849
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   cnpj                            56850 non-null  object 
 1   identificador_matriz_filial     56850 non-null  object 
 2   razao_social                    56850 non-null  object 
 3   situacao_cadastral              56850 non-null  object 
 4   data_inicio_atividade           56850 non-null  object 
 5   uf                              56850 non-null  object 
 6   codigo_municipio                56850 non-null  object 
 7   municipio                       56850 non-null  object 
 8   capital_social_empresa          56850 non-null  float64
 9   porte_empresa                   56850 non-null  object 
 10  opcao_pelo_simples              56850 non-null  object 
 11  cod_natureza_juridica           56850 non-null  object 
 12  nm_natureza_juridica            

In [73]:
dadosCadastroCnpj.to_csv('./dadosCadastroCnpj.csv', index = False)

## Criação do dataset
Até o momento obtivemos os dados que consideramos necessários para tentar prever se um determinado fornecedor PJ tem tendência a descumprir contratos, porém os dados ainda estão separados em 4 dataframes (_dadosCadastroCnpj_, _licitacoesPorFornecedor_,  _contratosPorFornecedor_ e _ocorrenciasPorFornecedor_).

Além disso, precisamos definir a variável de interesse. Para isso, utilizaremos os dados das ocorrências. De modo simplificado, uma ocorrência é registrada no sistema cada vez que o fornecedor descumpre alguma regra da licitação ou cláusula contratual. Sendo assim, definiremos o risco de cada empresa com base na existência ou não de ocorrências registradas em seu CNPJ.

Primeiro, juntamos os dataframes _dadosCadastroCnpj_, _licitacoesPorFornecedor_ e _contratosPorFornecedor_, que serão as variáveis preditivas:

In [74]:
dadosCadastroCnpj = pd.read_csv('./dadosCadastroCnpj.csv', dtype = 'str')
licitacoesPorFornecedor = pd.read_csv('./licitacoesPorFornecedor.csv', dtype = 'str')
contratosPorFornecedor = pd.read_csv('./contratosPorFornecedor.csv', dtype = 'str')

In [75]:
dataset = pd.merge(dadosCadastroCnpj, licitacoesPorFornecedor, left_on = 'cnpj', right_on = 'CNPJ', how = 'left')
dataset = pd.merge(dataset, contratosPorFornecedor, left_on = 'cnpj', right_on = 'CNPJ', how = 'left')

A seguir, juntamos os dados das ocorrências:

In [76]:
dataset = pd.merge(dataset, ocorrenciasPorFornecedor, left_on = 'cnpj', right_on = 'CNPJ', how = 'left')

In [77]:
dataset.head()

,cnpj,identificador_matriz_filial,razao_social,situacao_cadastral,data_inicio_atividade,uf,codigo_municipio,municipio,capital_social_empresa,porte_empresa,opcao_pelo_simples,cod_natureza_juridica,nm_natureza_juridica,cod_subclass_natureza_juridica,nm_subclass_natureza_juridica,cod_secao,nm_secao,cod_divisao,nm_divisao,cod_grupo,nm_grupo,cod_classe,nm_classe,cod_cnae,nm_cnae,CNPJ_x,participacoes,vitorias,CNPJ_y,contratos,CNPJ,ocorrencias
0,00000000000191,MATRIZ,BANCO DO BRASIL SA,Ativa,1966-08-01,DF,9701,BRASILIA,90000000000.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.22-1,Bancos múltiplos com carteira comercial,6422100,Bancos múltiplos com carteira comercial,00000000000191,6585,6580,00000000000191,67,00000000000191,8.0
1,00000000096369,FILIAL,BANCO DO BRASIL SA,Ativa,1975-08-11,RN,1619,APODI,0.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.21-2,Bancos comerciais,6421200,Bancos comerciais,00000000096369,1,1,00000000096369,1,NaN,NaN
2,00000000105104,FILIAL,BANCO DO BRASIL SA,Ativa,1976-03-17,RJ,6001,RIO DE JANEIRO,0.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.21-2,Bancos comerciais,6421200,Bancos comerciais,NaN,NaN,NaN,00000000105104,1,NaN,NaN
3,00000000304492,FILIAL,BANCO DO BRASIL SA,Ativa,1991-04-12,SP,6477,GUARULHOS,0.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.22-1,Bancos múltiplos com carteira comercial,6422100,Bancos múltiplos com carteira comercial,NaN,NaN,NaN,00000000304492,1,NaN,NaN
4,00000000314455,FILIAL,BANCO DO BRASIL SA,Ativa,1991-04-02,DF,9701,BRASILIA,0.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.22-1,Bancos múltiplos com carteira comercial,6422100,Bancos múltiplos com carteira comercial,NaN,NaN,NaN,00000000314455,1,NaN,NaN


Vamos excluir as colunas _CNPJ_x_, _CNPJ_y_ e _CNPJ_, que trazem a mesma informação que a coluna _cnpj_:

In [78]:
dataset.drop(['CNPJ_x', 'CNPJ_y', 'CNPJ'], axis = 1, inplace = True)

Verificando valores nulos:

In [79]:
dataset.isnull().sum()

cnpj                                  0
identificador_matriz_filial           0
razao_social                          0
situacao_cadastral                    0
data_inicio_atividade                 0
uf                                    0
codigo_municipio                      0
municipio                             0
capital_social_empresa                0
porte_empresa                         0
opcao_pelo_simples                    0
cod_natureza_juridica                 0
nm_natureza_juridica                  0
cod_subclass_natureza_juridica        0
nm_subclass_natureza_juridica         0
cod_secao                             0
nm_secao                              0
cod_divisao                           0
nm_divisao                            0
cod_grupo                             0
nm_grupo                              0
cod_classe                            0
nm_classe                             0
cod_cnae                              0
nm_cnae                               0


As linhas que contém valores nulos na coluna _ocorrencias_ são as empresas que não tem nenhuma ocorrência registrada no sistema, portanto, são as empresas com baixo risco de descumprimento contratual. As que contém valores não nulos, por outro lado, são as empresas com maior risco. Então podemos substituir estes valores nulos por zero.

In [80]:
dataset['ocorrencias'].fillna(0, inplace = True)

Quanto aos valores nulos nas colunas _participacoes_ e _vitorias_, se tratam de empresas que possuem contratos com o governo federal mas não participaram de nenhuma licitação. Esta é uma situação que pode parecer estranha, porém verificando no Portal da Transparência, realmente existem contratações "Sem Processo Licitatório Associado". Sendo assim, substituiremos os valores nulos nestas colunas por 0.

In [81]:
dataset['participacoes'].fillna(0, inplace = True)
dataset['vitorias'].fillna(0, inplace = True)

In [82]:
dataset.isnull().sum()

cnpj                              0
identificador_matriz_filial       0
razao_social                      0
situacao_cadastral                0
data_inicio_atividade             0
uf                                0
codigo_municipio                  0
municipio                         0
capital_social_empresa            0
porte_empresa                     0
opcao_pelo_simples                0
cod_natureza_juridica             0
nm_natureza_juridica              0
cod_subclass_natureza_juridica    0
nm_subclass_natureza_juridica     0
cod_secao                         0
nm_secao                          0
cod_divisao                       0
nm_divisao                        0
cod_grupo                         0
nm_grupo                          0
cod_classe                        0
nm_classe                         0
cod_cnae                          0
nm_cnae                           0
participacoes                     0
vitorias                          0
contratos                   

Agora, vamos criar a coluna do risco a partir da coluna _ocorrencias_, e depois excluir _ocorrencias_.

In [83]:
dataset['risco'] = dataset['ocorrencias'].apply(lambda x: 0 if x == 0 else 1)

In [84]:
dataset.drop(['ocorrencias'], axis = 1, inplace = True)

In [85]:
dataset.head()

,cnpj,identificador_matriz_filial,razao_social,situacao_cadastral,data_inicio_atividade,uf,codigo_municipio,municipio,capital_social_empresa,porte_empresa,opcao_pelo_simples,cod_natureza_juridica,nm_natureza_juridica,cod_subclass_natureza_juridica,nm_subclass_natureza_juridica,cod_secao,nm_secao,cod_divisao,nm_divisao,cod_grupo,nm_grupo,cod_classe,nm_classe,cod_cnae,nm_cnae,participacoes,vitorias,contratos,risco
0,00000000000191,MATRIZ,BANCO DO BRASIL SA,Ativa,1966-08-01,DF,9701,BRASILIA,90000000000.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.22-1,Bancos múltiplos com carteira comercial,6422100,Bancos múltiplos com carteira comercial,6585,6580,67,1
1,00000000096369,FILIAL,BANCO DO BRASIL SA,Ativa,1975-08-11,RN,1619,APODI,0.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.21-2,Bancos comerciais,6421200,Bancos comerciais,1,1,1,0
2,00000000105104,FILIAL,BANCO DO BRASIL SA,Ativa,1976-03-17,RJ,6001,RIO DE JANEIRO,0.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.21-2,Bancos comerciais,6421200,Bancos comerciais,0,0,1,0
3,00000000304492,FILIAL,BANCO DO BRASIL SA,Ativa,1991-04-12,SP,6477,GUARULHOS,0.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.22-1,Bancos múltiplos com carteira comercial,6422100,Bancos múltiplos com carteira comercial,0,0,1,0
4,00000000314455,FILIAL,BANCO DO BRASIL SA,Ativa,1991-04-02,DF,9701,BRASILIA,0.0,Demais,Não optante,2,Entidades Empresariais,2038,Sociedade de Economia Mista,K,ATIVIDADES FINANCEIRAS DE SEGUROS E SERVIÇOS R...,64,ATIVIDADES DE SERVIÇOS FINANCEIROS,64.2,Intermediação monetária - depósitos à vista,64.22-1,Bancos múltiplos com carteira comercial,6422100,Bancos múltiplos com carteira comercial,0,0,1,0


In [86]:
dataset.shape

(56850, 29)

In [87]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56850 entries, 0 to 56849
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   cnpj                            56850 non-null  object
 1   identificador_matriz_filial     56850 non-null  object
 2   razao_social                    56850 non-null  object
 3   situacao_cadastral              56850 non-null  object
 4   data_inicio_atividade           56850 non-null  object
 5   uf                              56850 non-null  object
 6   codigo_municipio                56850 non-null  object
 7   municipio                       56850 non-null  object
 8   capital_social_empresa          56850 non-null  object
 9   porte_empresa                   56850 non-null  object
 10  opcao_pelo_simples              56850 non-null  object
 11  cod_natureza_juridica           56850 non-null  object
 12  nm_natureza_juridica            56850 non-null

Criamos então um dataset com 56.850 fornecedores do Governo Federal, com 29 colunas, sendo a coluna _risco_ a que queremos prever com o modelo de ML.

In [89]:
dataset.to_csv('./dataset.csv', index = False)